# AstroGraphAnomaly — Colab Showcase (analyse + graphiques)

Objectif : démontrer au maximum les capacités du workflow :
- exécution offline (CSV test) + multi-engines
- comparaison de stabilité (overlap top anomalies)
- analyses stats (quantiles, entropie) + PCA
- analyses graphe (communautés, k-core, betweenness, articulation, bridges)
- visualisations (score hist, RA/Dec, CMD si bp_rp, graphe coloré)
- explainability (LIME) + lecture prompts LLM

Ce notebook détecte automatiquement l’entrypoint : `workflow.py` ou `run_workflow.py`.


In [ ]:
!git clone --depth 1 https://github.com/dalozedidier-dot/AstroGraphAnomaly.git
%cd AstroGraphAnomaly
!python -m pip install -q --upgrade pip
!pip -q install -r requirements.txt


## 0) Helpers (détection entrypoint + runner)

In [ ]:
import os, sys, subprocess, json
from pathlib import Path
import numpy as np
import pandas as pd

ENTRYPOINT = None
if Path('workflow.py').exists():
    ENTRYPOINT = 'workflow.py'
elif Path('run_workflow.py').exists():
    ENTRYPOINT = 'run_workflow.py'
else:
    raise FileNotFoundError('Aucun entrypoint trouvé: workflow.py ou run_workflow.py')
print('Entrypoint détecté:', ENTRYPOINT)

def run_job(mode: str, out_dir: str, **kw):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    if ENTRYPOINT == 'workflow.py':
        cmd = [sys.executable, ENTRYPOINT, mode]
        if mode in ('csv','hubble'):
            cmd += ['--in-csv', kw['in_csv']]
        if mode == 'gaia':
            cmd += ['--ra', str(kw['ra']), '--dec', str(kw['dec'])]
            cmd += ['--radius-deg', str(kw.get('radius_deg', 0.3)), '--limit', str(kw.get('limit', 800))]
        cmd += ['--out', str(out)]

        # options
        cmd += ['--engine', kw.get('engine','isolation_forest')]
        cmd += ['--threshold-strategy', kw.get('threshold_strategy','top_k')]
        cmd += ['--top-k', str(kw.get('top_k', 30))]
        cmd += ['--explain-top', str(kw.get('explain_top', 10))]
        cmd += ['--knn-k', str(kw.get('knn_k', 8))]
        cmd += ['--features-mode', kw.get('features_mode', 'extended')]
        if kw.get('plots', True):
            cmd += ['--plots']
    else:
        cmd = [sys.executable, ENTRYPOINT, '--mode', mode]
        if mode in ('csv','hubble'):
            cmd += ['--in-csv', kw['in_csv']]
        if mode == 'gaia':
            cmd += ['--ra', str(kw['ra']), '--dec', str(kw['dec'])]
            cmd += ['--radius-deg', str(kw.get('radius_deg', 0.3)), '--limit', str(kw.get('limit', 800))]
        cmd += ['--out', str(out)]

        cmd += ['--engine', kw.get('engine','isolation_forest')]
        cmd += ['--threshold-strategy', kw.get('threshold_strategy','top_k')]
        cmd += ['--top-k', str(kw.get('top_k', 30))]
        cmd += ['--explain-top', str(kw.get('explain_top', 10))]
        cmd += ['--knn-k', str(kw.get('knn_k', 8))]
        cmd += ['--features-mode', kw.get('features_mode', 'extended')]
        if kw.get('plots', True):
            cmd += ['--plots']

    print('RUN:', ' '.join(cmd))
    subprocess.check_call(cmd)
    return out

def load_outputs(out: Path):
    scored = pd.read_csv(out/'scored.csv')
    top = pd.read_csv(out/'top_anomalies.csv')
    return scored, top


## 1) Run showcase offline (CSV test fourni)
On lance plusieurs engines pour comparer la stabilité des anomalies et produire le maximum d’artefacts.

In [ ]:
DATA_CSV = 'data/sample_gaia_like.csv'
BASE_OUT = Path('results/showcase_offline')
BASE_OUT.mkdir(parents=True, exist_ok=True)

ENGINES = ['isolation_forest', 'lof', 'ocsvm', 'robust_zscore']
runs = {}
for eng in ENGINES:
    out = run_job(
        mode='csv',
        in_csv=DATA_CSV,
        out_dir=str(BASE_OUT/eng),
        engine=eng,
        threshold_strategy='top_k',
        top_k=30,
        explain_top=10,
        knn_k=8,
        features_mode='extended',
        plots=True,
    )
    runs[eng] = out
print('Done. Runs:', {k:str(v) for k,v in runs.items()})


## 2) Sanity + synthèse rapide (n, anomalies, quantiles, entropie)
Entropie calculée sur la distribution des labels (anomalie vs normal) et sur la distribution des scores (buckets).

In [ ]:
import math

def entropy_from_counts(counts):
    total = sum(counts)
    if total <= 0:
        return 0.0
    ent = 0.0
    for c in counts:
        if c <= 0:
            continue
        p = c/total
        ent -= p * math.log(p + 1e-12)
    return ent

summary_rows = []
for eng, out in runs.items():
    scored, top = load_outputs(out)
    n = len(scored)
    n_anom = int((scored['anomaly_label'] == -1).sum()) if 'anomaly_label' in scored.columns else None

    s = scored['anomaly_score'].to_numpy(float)
    q = np.quantile(s, [0,0.1,0.5,0.9,1.0]).tolist()

    # entropy of label distribution
    if n_anom is not None:
        ent_lbl = entropy_from_counts([n_anom, n-n_anom])
    else:
        ent_lbl = None
    # entropy of score histogram
    hist, _ = np.histogram(s, bins=20)
    ent_score = entropy_from_counts(hist.tolist())

    summary_rows.append({
        'engine': eng,
        'n': n,
        'n_anomalies(label=-1)': n_anom,
        'q0': q[0], 'q10': q[1], 'q50': q[2], 'q90': q[3], 'q100': q[4],
        'entropy_labels': ent_lbl,
        'entropy_score_hist': ent_score,
    })

summary = pd.DataFrame(summary_rows)
summary


## 3) Overlap des top anomalies (Jaccard)
On compare l’ensemble des `source_id` du top-k entre engines.

In [ ]:
tops = {}
for eng, out in runs.items():
    _, top = load_outputs(out)
    tops[eng] = set(top['source_id'].astype('int64').tolist())

eng_list = list(tops.keys())
J = np.zeros((len(eng_list), len(eng_list)), dtype=float)
for i,a in enumerate(eng_list):
    for j,b in enumerate(eng_list):
        inter = len(tops[a].intersection(tops[b]))
        union = len(tops[a].union(tops[b]))
        J[i,j] = inter/union if union else 0.0

J_df = pd.DataFrame(J, index=eng_list, columns=eng_list)
J_df


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,5))
plt.imshow(J, aspect='auto')
plt.xticks(range(len(eng_list)), eng_list, rotation=45, ha='right')
plt.yticks(range(len(eng_list)), eng_list)
plt.title('Jaccard overlap of top anomalies (top_k=30)')
plt.colorbar(label='Jaccard')
plt.tight_layout()
plt.show()


## 4) Visualisations comparatives des scores
- histogrammes par engine
- scatter RA/Dec coloré par score (engine choisi)
- PCA 2D (features numériques) avec anomalies surlignées

In [ ]:
plt.figure(figsize=(9,6))
for eng, out in runs.items():
    scored, _ = load_outputs(out)
    plt.hist(scored['anomaly_score'].to_numpy(float), bins=60, alpha=0.45, label=eng)
plt.title('Score distributions (overlay)')
plt.xlabel('anomaly_score'); plt.ylabel('count')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Choisir un engine pour les visualisations spatiales
ENGINE_VIEW = 'isolation_forest'
scored, top = load_outputs(runs[ENGINE_VIEW])

plt.figure(figsize=(9,6))
plt.scatter(scored['ra'], scored['dec'], c=scored['anomaly_score'], s=18, alpha=0.85)
plt.colorbar(label='anomaly_score')
plt.title(f'RA/Dec colored by score ({ENGINE_VIEW})')
plt.xlabel('ra'); plt.ylabel('dec')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Numeric feature block from scored.csv (auto)
num_cols = [c for c in scored.columns if c not in ('source_id') and pd.api.types.is_numeric_dtype(scored[c])]
X = scored[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0).to_numpy(float)
pca = PCA(n_components=2, random_state=42)
Z = pca.fit_transform(X)

is_anom = scored['anomaly_label'].to_numpy(int) == -1 if 'anomaly_label' in scored.columns else np.zeros(len(scored), dtype=bool)

plt.figure(figsize=(9,6))
plt.scatter(Z[~is_anom,0], Z[~is_anom,1], s=10, alpha=0.5)
plt.scatter(Z[is_anom,0], Z[is_anom,1], s=22, alpha=0.9)
plt.title(f'PCA(2) on scored numeric columns ({ENGINE_VIEW})')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.tight_layout()
plt.show()

print('Explained variance ratio:', pca.explained_variance_ratio_.tolist())


## 5) Analyse graphe (GraphML)
On recharge `graph_full.graphml` et on calcule des métriques : degree, clustering, k-core, betweenness (approx), communautés, articulation, bridges.
Puis on visualise :
- distribution degree/kcore/betweenness
- tailles de communautés
- graphe top-k coloré par communauté et anomalies

In [ ]:
import networkx as nx

G_full = nx.read_graphml(runs[ENGINE_VIEW]/'graph_full.graphml')
G_top = nx.read_graphml(runs[ENGINE_VIEW]/'graph_topk.graphml')
print('G_full:', G_full.number_of_nodes(), 'nodes', G_full.number_of_edges(), 'edges')
print('G_top :', G_top.number_of_nodes(), 'nodes', G_top.number_of_edges(), 'edges')


In [ ]:
# Convert node ids to a stable list (GraphML often loads as str)
nodes = list(G_full.nodes())
deg = dict(G_full.degree())
clust = nx.clustering(G_full)
core = nx.core_number(G_full) if G_full.number_of_nodes() > 0 else {n:0 for n in nodes}

# betweenness approx (k limited)
k = min(300, len(nodes))
btw = nx.betweenness_centrality(G_full, k=k, normalized=True, seed=42) if len(nodes) > 1 else {n:0.0 for n in nodes}

# communities (Louvain if available)
try:
    from networkx.algorithms.community import louvain_communities
    comms = louvain_communities(G_full, seed=42)
except Exception:
    from networkx.algorithms.community import greedy_modularity_communities
    comms = greedy_modularity_communities(G_full)

comm_id = {}
for i, cset in enumerate(comms):
    for n in cset:
        comm_id[n] = i

aps = set(nx.articulation_points(G_full)) if G_full.number_of_nodes() > 2 else set()
try:
    bridges = list(nx.bridges(G_full))
    bridge_nodes = set([a for a,b in bridges] + [b for a,b in bridges])
except Exception:
    bridge_nodes = set()

gdf = pd.DataFrame({
    'node': nodes,
    'degree': [deg.get(n,0) for n in nodes],
    'clustering': [clust.get(n,0.0) for n in nodes],
    'kcore': [core.get(n,0) for n in nodes],
    'betweenness': [btw.get(n,0.0) for n in nodes],
    'community': [comm_id.get(n,-1) for n in nodes],
    'is_articulation': [1 if n in aps else 0 for n in nodes],
    'incident_to_bridge': [1 if n in bridge_nodes else 0 for n in nodes],
})
gdf.head()


In [ ]:
plt.figure(figsize=(9,6))
plt.hist(gdf['degree'], bins=50, alpha=0.8)
plt.title('Degree distribution')
plt.xlabel('degree'); plt.ylabel('count')
plt.tight_layout(); plt.show()

plt.figure(figsize=(9,6))
plt.hist(gdf['kcore'], bins=30, alpha=0.8)
plt.title('k-core distribution')
plt.xlabel('kcore'); plt.ylabel('count')
plt.tight_layout(); plt.show()

plt.figure(figsize=(9,6))
plt.hist(gdf['betweenness'], bins=60, alpha=0.8)
plt.title('Betweenness distribution (approx)')
plt.xlabel('betweenness'); plt.ylabel('count')
plt.tight_layout(); plt.show()


In [ ]:
# Community sizes
cs = gdf.groupby('community').size().sort_values(ascending=False)
plt.figure(figsize=(10,5))
plt.bar(range(len(cs.head(25))), cs.head(25).to_numpy())
plt.title('Top community sizes (up to 25)')
plt.xlabel('community rank'); plt.ylabel('size')
plt.tight_layout(); plt.show()
cs.head(10)


In [ ]:
# Plot top-k graph colored by community, anomalies highlighted
top_ids = set(top['source_id'].astype(str).tolist())
anom_ids = set(scored.loc[scored['anomaly_label']==-1,'source_id'].astype(str).tolist()) if 'anomaly_label' in scored.columns else set()

pos = nx.spring_layout(G_top, seed=42)

node_list = list(G_top.nodes())
node_colors = []
sizes = []
for n in node_list:
    c = comm_id.get(n, 0)
    node_colors.append(c)
    sizes.append(55 if n in anom_ids else 20)

plt.figure(figsize=(10,8))
nx.draw(G_top, pos, node_size=sizes, node_color=node_colors, with_labels=False, alpha=0.85, width=0.4)
plt.title('Top-k subgraph colored by community (larger = anomaly)')
plt.tight_layout(); plt.show()


## 6) Explainability (LIME) + prompts LLM
On lit `explanations.jsonl` et `llm_prompts.jsonl` (si présents), puis on visualise les poids LIME du premier élément.

In [ ]:
import json

exp_path = runs[ENGINE_VIEW] / 'explanations.jsonl'
prm_path = runs[ENGINE_VIEW] / 'llm_prompts.jsonl'
print('explanations exists:', exp_path.exists())
print('prompts exists:', prm_path.exists())

first_exp = None
if exp_path.exists():
    with exp_path.open('r', encoding='utf-8') as f:
        first_exp = json.loads(next(f))
    first_exp.keys(), first_exp.get('source_id')


In [ ]:
if first_exp is not None:
    weights = first_exp.get('lime', {}).get('weights', [])
    if weights:
        # take top 12 by abs(weight)
        weights = sorted(weights, key=lambda w: abs(w.get('weight',0.0)), reverse=True)[:12]
        labels = [w['feature'] for w in weights]
        vals = [w['weight'] for w in weights]
        plt.figure(figsize=(10,5))
        plt.bar(range(len(vals)), vals)
        plt.xticks(range(len(vals)), labels, rotation=45, ha='right')
        plt.title('LIME weights (first explained anomaly)')
        plt.tight_layout(); plt.show()
    else:
        print('No LIME weights (lime missing or fallback).')


In [ ]:
if prm_path.exists():
    with prm_path.open('r', encoding='utf-8') as f:
        obj = json.loads(next(f))
    print('Prompt sample for source_id:', obj.get('source_id'))
    print(obj.get('prompt')[:1600])
else:
    print('No prompts file found.')


## 7) Optionnel : Gaia run (réseau requis)
Décommenter pour exécuter Gaia et obtenir `bp_rp` + CMD (si activé dans la requête).

In [ ]:
# out_gaia = run_job(
#     mode='gaia',
#     ra=266.4051, dec=-28.936175,
#     radius_deg=0.3,
#     limit=800,
#     out_dir='results/showcase_gaia/isolation_forest',
#     engine='isolation_forest',
#     threshold_strategy='top_k',
#     top_k=30,
#     explain_top=10,
#     knn_k=8,
#     features_mode='extended',
#     plots=True,
# )
# print('Gaia outputs in:', out_gaia)
